In [1]:
import os, re 
import numpy as np
import tensorflow as tf

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 총개수 187088개

In [3]:
len(raw_corpus)

187088

# 전처리

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end> '
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end> 


### 정제한 결과 14120개

In [15]:
corpus = []


for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence) > 15: continue  # 공백 포함해서 합계 15 초과는 배제
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        

print(len(corpus[:])) # 정제한 결과는 14120개이다.
print(corpus[:10])

14120
['<start> hallelujah <end> ', '<start> hallelujah <end> ', '<start> she tied you <end> ', '<start> hallelujah <end> ', '<start> hallelujah <end> ', '<start> in every word <end> ', '<start> hallelujah <end> ', '<start> hallelujah <end> ', '<start> and even though <end> ', '<start> hallelujah <end> ']


### num_words에 14120개 기입

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14120,  
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 162   3 ...   0   0   0]
 [  2 162   3 ...   0   0   0]
 [  2  56 881 ...   0   0   0]
 ...
 [  2 216 554 ...   0   0   0]
 [  2  60  17 ...   0   0   0]
 [  2  36  21 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f8eb7a58f10>


In [17]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])



[  2 162   3   0   0   0   0   0   0   0   0   0]
[162   3   0   0   0   0   0   0   0   0   0   0]


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

# 학습

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 12, 14121), dtype=float32, numpy=
array([[[ 8.58998392e-05,  2.89846266e-05,  6.92937829e-06, ...,
         -1.09919289e-04,  1.15501789e-04, -3.58508150e-05],
        [-2.08016590e-05, -3.95828238e-06, -9.97651296e-05, ...,
         -2.42006630e-04,  1.35944283e-04,  2.73783517e-04],
        [ 2.69058182e-05,  4.97728142e-05, -2.92374636e-04, ...,
         -1.63932069e-04,  6.57883793e-05,  5.17553475e-04],
        ...,
        [-3.75499105e-04, -2.42668626e-04, -1.00314990e-03, ...,
          1.74755976e-03, -1.62696710e-03, -2.18058727e-03],
        [-5.52661368e-04, -4.92478663e-04, -8.91218719e-04, ...,
          2.16150866e-03, -1.97396544e-03, -2.69659143e-03],
        [-6.92180649e-04, -7.55017332e-04, -7.57718342e-04, ...,
          2.53119576e-03, -2.26226193e-03, -3.13537265e-03]],

       [[ 8.58998392e-05,  2.89846266e-05,  6.92937829e-06, ...,
         -1.09919289e-04,  1.15501789e-04, -3.58508150e-05],
        [-2.08016590e-05, -3.95828238e-06, -9

In [11]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3614976   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  14474025  
Total params: 31,728,681
Trainable params: 31,728,681
Non-trainable params: 0
_________________________________________________________________


In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

model.fit(dataset, epochs=3)

Epoch 1/3
55/55 [==============================] - 11s 165ms/step - loss: 2.8643
Epoch 2/3
55/55 [==============================] - 9s 171ms/step - loss: 1.7553
Epoch 3/3
55/55 [==============================] - 9s 167ms/step - loss: 1.5915


In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> it")

'<start> it <end> '